# Result Analysis

Setting up useful variables and external libraries

In [1]:
#%pdb on

import os
import pandas as pd

pth = '<score-charts-pth>'

## Data Understanding 

This table shows an example of game result.  

In [2]:
score_file = '<score-xls-file>' # Ex: 'score-nonstat_ne0.2_ge0.2_pg0.1_pa1.0.xls'
df = pd.read_excel(os.path.join(pth, ), header=0, index_col=0)
df

,Exp3,FreqHist,Frequentist,Nash,NoRegret,Reply-Last,ReplyHist,Skynet,UCB1-Tuned,Xelnaga,e-Greedy,e-Nash
Exp3,NaN,43.460000,85.726667,73.430000,73.623333,81.790000,44.666667,66.413333,46.216667,71.713333,48.460000,74.206667
FreqHist,56.540000,NaN,91.090000,73.443333,71.020000,91.540000,49.490000,49.913333,51.403333,49.826667,54.506667,67.740000
Frequentist,14.273333,8.910000,NaN,18.913333,35.160000,36.593333,9.546667,31.800000,29.106667,8.300000,15.223333,18.413333
Nash,26.570000,26.556667,81.086667,NaN,50.543333,64.283333,28.513333,25.710000,25.253333,45.720000,35.770000,56.366667
NoRegret,26.376667,28.980000,64.840000,49.456667,NaN,58.313333,28.720000,26.163333,25.593333,45.020000,35.346667,52.093333
Reply-Last,18.210000,8.460000,63.406667,35.716667,41.686667,NaN,8.726667,31.886667,25.146667,8.260000,17.623333,30.890000
ReplyHist,55.333333,50.510000,90.453333,71.486667,71.280000,91.273333,NaN,52.720000,50.910000,50.383333,55.523333,70.996667
Skynet,33.586667,50.086667,68.200000,74.290000,73.836667,68.113333,47.280000,NaN,29.393333,76.100000,33.550000,72.593333
UCB1-Tuned,53.783333,48.596667,70.893333,74.746667,74.406667,74.853333,49.090000,70.606667,NaN,75.233333,54.480000,73.516667
Xelnaga,28.286667,50.173333,91.700000,54.280000,54.980000,91.740000,49.616667,23.900000,24.766667,NaN,31.020000,61.230000


The line/column Exp3/FreqHist has the value 43.46. This represents the percentage of winning Exp3 has over FreqHist on 1000 rounds of a particular simulation.

The result file has the defined configuration on its name. The file is intepreted as follows:

- `nonstat`: represents if the game is stationary or non-stationary (possible values are `nonstat` and `stat`);
- `ne#`: $\varepsilon$ value of $\varepsilon$-Nash (`ne0.2` means $\varepsilon = 0.2$);
- `ge#`: $\epsilon$ value of $\epsilon$-Greedy;
- `pg#` and `pa#`: $\gamma$ and $\alpha$ of Exp3, respectively.

## Analysing the Data

Defining helper functions

In [3]:
def mean_players(xls_file, players):
    df = pd.read_excel(xls_file, header=0, index_col=0)
    return {(pname + '_mean'): df.loc[pname].mean() for pname in players}

In [4]:
def sim_conf(xls_name):
    fname = os.path.splitext(xls_name)[0]
    fname = fname.replace('score-', '')
    pars = fname.split('_')
    dpars = {p[:2]: float(p[2:]) for p in pars[1:]}
    dpars['game_type'] = pars[0]
    return dpars

In [5]:
def best_conf(df, player_mean, game_type):
    gdf = df[df['game_type'] == game_type]
    idx = gdf[player_mean].idxmax()
    return gdf.loc[idx]

Here we calculate the average winning rate of a set of players (Exp3, $\varepsilon$-Nash and $\epsilon$-Greedy, in this case)

In [6]:
all_data = []

for f in os.listdir(pth):
    if f.endswith('.xls'):
        dpars = sim_conf(f)
        dmeans = mean_players(os.path.join(pth, f), ['Exp3', 'e-Greedy', 'e-Nash'])
        all_data.append(dict(dpars.items() | dmeans.items()))
        
df_data = pd.DataFrame(all_data)
df_data

,Exp3_mean,e-Greedy_mean,e-Nash_mean,game_type,ge,ne,pa,pg
0,61.198788,62.361818,41.607273,nonstat,0.2,0.2,0.318103,0.159052
1,63.481818,62.190303,41.605152,nonstat,0.2,0.2,0.600000,0.159052
2,64.518788,62.120000,41.557273,nonstat,0.2,0.2,1.000000,0.100000
3,60.393333,62.406970,41.740000,nonstat,0.2,0.2,0.318103,0.300000
4,61.529091,62.210606,41.606667,nonstat,0.2,0.2,0.600000,0.300000
5,59.441818,62.881818,41.637576,nonstat,0.2,0.2,1.000000,0.400000
6,54.363939,62.945455,42.423030,nonstat,0.2,0.2,0.318103,0.600000
7,54.519697,63.152424,42.523939,nonstat,0.2,0.2,0.600000,0.600000
8,61.535455,58.567879,42.008182,nonstat,0.4,0.2,0.318103,0.159052
9,63.615152,58.373939,42.013636,nonstat,0.4,0.2,0.600000,0.159052


We find the best configuration parameters in each game type (stationary or nonstationary) for this set of players by finding the highest win rate of each player in the above table. This will give us the configuration parameters witch leaded each player to its best performance.

These parameters don’t necessarily give the best performance for all players in all cases (this depends of the opponent's configuration), but it's still a reasonable estimate of good parameters for each player.

In [7]:
ns1 = best_conf(df_data, 'Exp3_mean', 'nonstat')[['Exp3_mean', 'pa', 'pg']]
ns2 = best_conf(df_data, 'e-Greedy_mean', 'nonstat')[['e-Greedy_mean', 'ge']]
ns3 = best_conf(df_data, 'e-Nash_mean', 'nonstat')[['e-Nash_mean', 'ne']]
nspd = pd.DataFrame(pd.concat([ns1,ns2,ns3], axis=0)).transpose()

s1 = best_conf(df_data, 'Exp3_mean', 'stat')[['Exp3_mean', 'pa', 'pg']]
s2 = best_conf(df_data, 'e-Greedy_mean', 'stat')[['e-Greedy_mean', 'ge']]
s3 = best_conf(df_data, 'e-Nash_mean', 'stat')[['e-Nash_mean', 'ne']]
spd = pd.DataFrame(pd.concat([s1,s2,s3], axis=0)).transpose()

dr = pd.concat([spd, nspd])
dr.index = ['stat', 'nonstat']
dr

,Exp3_mean,pa,pg,e-Greedy_mean,ge,e-Nash_mean,ne
stat,55.5315,1,0.1,53.0333,0.2,53.0879,0.4
nonstat,65.0009,1,0.1,63.9348,0.2,42.7979,0.2
